**Zero Redundancy Optimizer（ZeRO）** 是现代大模型训练的核心技术之一，尤其是在分布式训练中非常重要
---

## 🚀 一、问题背景：为什么需要 ZeRO？

在训练大模型（如 GPT、BERT、LLM）时，
模型参数数量非常庞大（上亿、上百亿甚至上万亿），
单个 GPU 无法同时容纳：

* 模型参数（model weights）
* 优化器状态（optimizer states）
* 梯度（gradients）

例如，一个 10B 参数的模型：

> 仅优化器状态（如 Adam 的动量和方差）
> 就可能占用 **参数的 4~8 倍显存！**

→ 所以传统的 **Data Parallel (DP)** 已经无法负担。

---

## 💡 二、ZeRO 的核心思想

**ZeRO (Zero Redundancy Optimizer)** 的目标是：

> **在多 GPU 上去除冗余的模型状态存储，让每个 GPU 只保存必要的一部分。**

传统的 Data Parallel：

* 每个 GPU 都保存一整份参数、梯度、优化器状态；
* 很浪费显存（N 倍冗余）。

ZeRO：

* 把这些状态**分片（shard）**到不同 GPU；
* 每个 GPU 只保存自己负责的那一部分；
* 需要计算或更新时再彼此通信（AllGather/ReduceScatter）。

---

## 🧩 三、ZeRO 的三个阶段（Stage 1 / 2 / 3）

| 阶段         | 优化器状态 | 梯度    | 参数    | 内存节省 | 通信代价 |
| ---------- | ----- | ----- | ----- | ---- | ---- |
| **ZeRO-1** | ✅ 分片  | ❌ 不分片 | ❌ 不分片 | 中等   | 低    |
| **ZeRO-2** | ✅ 分片  | ✅ 分片  | ❌ 不分片 | 高    | 中    |
| **ZeRO-3** | ✅ 分片  | ✅ 分片  | ✅ 分片  | 最高   | 高    |

### ✅ ZeRO-1

* 优化器状态分布在多个 GPU 上；
* 每个 GPU 只保存一部分动量和方差；
* 参数和梯度仍然是完整的。

📉 显存节省：约 4×（Adam）的一部分。

---

### ✅ ZeRO-2

* 再进一步，把 **梯度** 也分片；
* 所以反向传播时，梯度在计算完会直接分布式平均，不会在每个 GPU 上保存完整副本。

📉 显存进一步下降，但需要更多通信（ReduceScatter）。

---

### ✅ ZeRO-3

* 最彻底的方案；
* 连 **模型参数本身** 也分片，每个 GPU 只保存自己负责的那部分参数；
* 在前向和反向时，临时 gather 需要的参数块。

📉 显存使用最少，**可以训练万亿参数级模型**；
📈 通信最频繁，但 DeepSpeed 做了高效调度优化。

---

## 🔧 四、架构类比（简单理解）

假设模型参数有 12 个，
传统 Data Parallel 每个 GPU 都保存全部：

```
GPU0: [1,2,3,4,5,6,7,8,9,10,11,12]
GPU1: [1,2,3,4,5,6,7,8,9,10,11,12]
GPU2: [1,2,3,4,5,6,7,8,9,10,11,12]
```

→ 3 倍冗余。

ZeRO 把它们分片：

```
GPU0: [1,2,3,4]
GPU1: [5,6,7,8]
GPU2: [9,10,11,12]
```

→ 每个 GPU 只存 1/3 参数；
→ 显存减少到原来的 1/3；
→ 需要计算时再 AllGather。

---

## ⚙️ 五、ZeRO 实现框架

ZeRO 是 **DeepSpeed**（微软开源）的一部分，
也被 **PyTorch Fully Sharded Data Parallel (FSDP)** 吸收进去了。

| 框架          | 对应技术                          |
| ----------- | ----------------------------- |
| DeepSpeed   | ZeRO                          |
| PyTorch     | FSDP（功能上类似 ZeRO）              |
| Megatron-LM | ZeRO + Model Parallel 结合      |
| ColossalAI  | Hybrid ZeRO + Tensor Parallel |

---

## 📊 六、显存节省对比（Adam 优化器）

假设模型参数大小为 `P`：

| 组成部分         | 普通DP占用 | ZeRO-3占用 |
| ------------ | ------ | -------- |
| 参数           | P      | P / N    |
| 梯度           | P      | P / N    |
| 优化器状态（动量+方差） | 4P     | 4P / N   |
| **总计**       | 6P     | 6P / N   |

N 是 GPU 数量。
→ 例如 8 张 GPU，可节省 87.5% 显存！

---

## 🧠 七、一句话总结

> **ZeRO 通过在多 GPU 间分片参数、梯度和优化器状态，实现显存零冗余，从而让超大模型也能在有限显存上训练。**

---

## 💬 附加说明：名字含义

* “Zero” = “Zero Redundancy”（零冗余）
* “Optimizer” = 它最初优化的对象是优化器状态（stage 1）

---
